<a href="https://colab.research.google.com/github/rmaacario/spatial-semantics-translation/blob/main/Computing_Scores_for_NMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate BLUE, METEOR, BERTscore and COMET scores for MT

In [ ]:
!pip install unbabel-comet
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadat

In [ ]:
# Read the excel data
path = "/content/TEDTalks.en_pt-br.llama3.csv"

In [ ]:
import pandas as pd


df = pd.read_csv(path)
df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)
df.rename(columns={'target': 'reference'}, inplace=True)
df
#df.rename(columns={'Amazon custom': 'amazon-custom'}, inplace=True)

,lang_pair,source,reference,llama3,inner_id,produced_from
0,en_pt_br,So we went over into Michael's garage — his da...,Fomos na garagem de Michael — seu pai tinha to...,Então vamos até o garaje de Michael – o pai de...,548672,SRC_TM_DATA/data_2024/en-pt_br.tmx
1,en_pt_br,"I look out into this crowd, and I see 400 peop...",Eu olho para essa multidão e vejo 400 pessoas ...,Olho para esta multidão e vejo 400 pessoas que...,121062,SRC_TM_DATA/data_2024/en-pt_br-2.tmx
2,en_pt_br,"In fact, part of the charm and the innovation ...",Parte do charme e da inovação do trabalho real...,"Na verdade, parte do charme e da inovação dess...",142335,SRC_TM_DATA/data_2024/en-pt_br-2.tmx
3,en_pt_br,So you might be wondering whether we can also ...,Você deve estar perguntando se podemos também ...,Aí você pode estar se perguntando se podemos t...,157346,SRC_TM_DATA/data_2024/en-pt_br-2.tmx
4,en_pt_br,But there's also the fact that law is another ...,Mas há também o fato de que a lei é outro luga...,Mas também há o fato de que a lei é outro luga...,76527,SRC_TM_DATA/data_2024/en-pt_br-2.tmx
...,...,...,...,...,...,...
1995,en_pt_br,We're now moving into East Africa.,Agora estamos expandindo para a África Oriental.,Agora estamos mudando para África Oriental.,269204,SRC_TM_DATA/data_2024/en-pt_br-2.tmx
1996,en_pt_br,I had just gone through a breakup.,Eu tinha acabado de terminar um namoro.,Eu havia apenas passado por um fim de relacion...,108436,SRC_TM_DATA/data_2024/en-pt_br-2.tmx
1997,en_pt_br,Facit fell into the success trap.,A Facit caiu na armadilha do sucesso.,Caiu no trap da sucesso.,200436,SRC_TM_DATA/data_2024/en-pt_br-2.tmx
1998,en_pt_br,They looked straight through me.,Olharam direto para mim.,Eles olharam direto em mim.,232428,SRC_TM_DATA/data_2024/en-pt_br-2.tmx


In [ ]:
#df_new = df[~df['spatial_sense'].isna()]
#df_new

In [ ]:
#df = df[['lang_pair', 'source', 'reference', 'mixtral', 'inner_id', 'produced_from']]
#df.rename(columns={'target': 'reference'}, inplace=True)

In [ ]:
df['llama3'][4]

'Mas também há o fato de que a lei é outro lugar onde estão em circulação muitas opiniões, mas elas precisam ser resolvidas para uma cópia canônica única, e quando você vai ao GitHub e olha ao redor, há milhões e milhões de projetos, quase todos os quais são código fonte, mas se você olhar nos cantos, pode ver pessoas experimentando com as consequências políticas de um sistema como esse.'

In [ ]:
import re

df['llama3'] = df.apply(lambda row: re.sub('^\"|\"$', '', row['llama3']), axis=1)
df['llama3'] = df.apply(lambda row: re.sub('\n\n', ' ', row['llama3']), axis=1)
df['llama3'] = df['llama3'].apply(lambda x: x.strip())

In [ ]:
df['llama3'] = df['llama3'].apply(lambda x: x[:-3] + '.' if x.endswith('') else x)

In [ ]:
df['llama3'] = df['llama3'].str.replace("L", '')

In [ ]:
df['mistral'] = df['mistral'].apply(lambda x: x.replace('', '').rstrip(''))

In [ ]:
df['llama3'] = df['llama3'].str.split("Breakdown ").str[0]
#df['llama3'] = df['llama3'].str.split("Note: ").str[0]

In [ ]:
# Drop " at the end of segments
#df['llama2:13b'] = df['llama2:13b'].str.rstrip('"')

In [ ]:
df.to_csv(path)

In [ ]:
#empty_rows = df[df['mixtral'].isna()]

In [ ]:

import re

# Define the regex pattern to match the given format
patterns = [ r'\(Note:.*?\)|\([^()]*\)',
             r'NOTE:.*',
             r'Mas note que.*',
             r'Em português:.*',
             r'Em português,.*',
             r'Em tradução livre,.*',
             r'Observação:.*',
             r'Nota:.*',
             r'Obs:',
             r'Note:.*',
             r'Confira a tradução.*',
             r'Confirmação:.*',
             r'A tradução.*',
             r'In this context,.*',
             r'Please note.*',
             r'In Portuguese.*',
             r'\n.*',
             r'This translation.*',
             r"t's important to note.* ",
             r'In this sentence,.*',
             r'Explicação.*',
             r'Em Portugal,.* ',
             r'Explanation:.*',
             r"It's important to note.*",
             r'The Portuguese translation.*',
             r'Explicação:.*',
             r'The correct translation.*',
             r'O texto diz respeito.*',
             r'Confira também:.*',
             r'Aqui está a tradução do.*',
             r'Para sua referência,.*',
             r'O texto refere-se.*',
             r'O texto em português.*'
             ]

# Join the patterns using the '|' operator to create a single regex pattern
combined_pattern = '|'.join(patterns)

# Remove text enclosed in parentheses from the column
df['llama3'] = df['llama3'].str.replace(combined_pattern, '', regex=True)

In [ ]:
import string

# Function to check if a segment ends with punctuation and add a period if needed
def add_period(segment):
    if segment[-1] not in string.punctuation:
        return segment + "."
    else:
        return segment

# Apply the function to the DataFrame column
df['llama3'] = df['llama3'].apply(add_period)

In [ ]:
import string

# Define punctuation characters
punctuation_chars = set(string.punctuation)

# Function to add a period at the end of segments if needed
def add_period(segment):
    if segment[-1] not in punctuation_chars:
        return segment + "."
    else:
        return segment

# Apply the function to the DataFrame column
df['mixtral'] = df['mixtral'].apply(lambda x: add_period(x))

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.5 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from comet import download_model, load_from_checkpoint
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import sacrebleu
import nltk
import zipfile
from google.colab import files

In [ ]:
# Download NLTK resources
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Aug 16 14:14:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
def read_from_folder(folder_path):

    dataframes = {}

    for csv_file in [file for file in os.listdir(folder_path) if file.endswith('.csv')]:
        file_path = os.path.join(folder_path, csv_file)
        print(f"Reading data from '{file_path}':")
    return csv_file

## Load .csv files from folder:

## Calculate BLUE and METEOR:

In [ ]:
def compute_scores(df, bleu_score_column, meteor_score_column):
    def calculate_scores(row):
        reference_text = row['reference']
        translation_text = row.iloc[3]
        reference_tok = [word_tokenize(reference_text)]
        translation_tok = word_tokenize(translation_text)

        # Calculate BLEU score using 4-grams
        bleu_score = sentence_bleu(reference_tok, translation_tok, smoothing_function=SmoothingFunction().method1)

        # Calculate METEOR score
        meteor = meteor_score(reference_tok, translation_tok)

        return bleu_score, meteor

    df[[bleu_score_column, meteor_score_column]] = df.apply(lambda row: pd.Series(calculate_scores(row)), axis=1)

### Calculate TER:

In [ ]:
def calculate_ter_scores(df, ter_score_column):
    def compute_ter_score(row):
        reference_text = row['reference']
        translation_text = row.iloc[3]
        ter_score = sacrebleu.sentence_ter(translation_text, [reference_text]).score
        return ter_score

    df[ter_score_column] = df.apply(compute_ter_score, axis=1)

### Calculate BERTscore:

In [ ]:
def calculate_bert_score(df, bert_score_column):
    # Load the pre-trained BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")

    def compute_bert_score(row):
        reference_text = row['reference']
        translation_text = row.iloc[3]

        # Tokenize the texts
        inputs_ref = tokenizer(reference_text, return_tensors="pt", padding=True, truncation=True)
        inputs_trans = tokenizer(translation_text, return_tensors="pt", padding=True, truncation=True)

        # Feed the texts to the BERT model
        outputs_ref = model(**inputs_ref)
        outputs_trans = model(**inputs_trans)

        # Obtain the representation vectors
        embeddings_ref = outputs_ref.last_hidden_state.mean(dim=1).detach().numpy()
        embeddings_trans = outputs_trans.last_hidden_state.mean(dim=1).detach().numpy()

        # Calculate cosine similarity
        similarity_score = np.dot(embeddings_trans, embeddings_ref.T) / (
                np.linalg.norm(embeddings_trans) * np.linalg.norm(embeddings_ref))

        return similarity_score[0][0]

    df[bert_score_column] = df.apply(compute_bert_score, axis=1)

### Calculate COMET:

In [ ]:
def calculate_comet_score(df, comet_score_column):
    model = load_from_checkpoint(download_model("Unbabel/wmt22-comet-da"))

    def compute_comet_score(row):
        data = {"src": row['source'],
                "mt": row.iloc[3],
                "ref": row['reference']}
        score = model.predict([data], batch_size=8, gpus=1)[1]
        return score

    df[comet_score_column] = df.apply(compute_comet_score, axis=1)

In [ ]:
import pandas as pd

# Creating the dataframe
df = pd.DataFrame({
    'languages': ['en-pt'],
    'source': ["Local communities on the East Coast would get together and whole teams of 10, 20 people would caravan across the United States, and they would form companies."],
    'reference': ['Pequenas comunidades do Leste se uniriam em equipes de 10 ou 20 pessoas e atravessariam o país, e formariam empresas.'],
    'gpt': ['Comunidades locais na Costa Leste se reuniriam e equipes inteiras de 10, 20 pessoas viajariam pelos Estados Unidos em caravana, formando empresas.']
})

df

,languages,source,reference,gpt
0,en-pt,Local communities on the East Coast would get ...,Pequenas comunidades do Leste se uniriam em eq...,Comunidades locais na Costa Leste se reuniriam...


## Compute all scores:

In [ ]:
def calculate_all_scores(df, bleu_score_column, meteor_score_column, bert_score_column, comet_score_column, ter_score_column):
    compute_scores(df, bleu_score_column, meteor_score_column)
    calculate_bert_score(df, bert_score_column)
    calculate_comet_score(df, comet_score_column)
    calculate_ter_scores(df, ter_score_column)

In [ ]:
#path = '/content/TEDTalks.en_pt-br.llama2_7B.csv'

In [ ]:
# Calculate scores for the DataFrame
#df = pd.read_csv(path)
#df.drop(columns='Unnamed: 0', inplace=True)
df
path = '/content/'

In [ ]:
calculate_all_scores(df, 'bleu_score', 'meteor_score', 'bert_score', 'comet_score', 'ter_score')
#filename = path.split('/')[-1]
#print(f"Scores calculated for '{filename}'.")

# Export the DataFrame to the same CSV file overwriting the original
#df.to_csv(path, index=False)
#files.download(path)
print(df)
#print("DataFrame exported to", path)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 11.53it/s]


  languages                                             source  \
0     en-pt  Local communities on the East Coast would get ...   

                                           reference  \
0  Pequenas comunidades do Leste se uniriam em eq...   

                                                 gpt  bleu_score  \
0  Comunidades locais na Costa Leste se reuniriam...     0.03546   

   meteor_score  bert_score  comet_score  ter_score  
0      0.486242    0.968044      0.85129       80.0  


In [ ]:
df

,languages,source,reference,deepl,bleu_score,meteor_score,bert_score,comet_score,ter_score
0,en-pt,He swam across the river.,Ele atravessou o rio nadando.,Ele nadou através do rio.,0.053728,0.25,0.873543,0.839464,80.0


In [ ]:
# Export the DataFrame to the same CSV file overwriting the original
df.to_csv(path, index=False)
files.download(path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,lang_pair,source,reference,llama3,inner_id,produced_from,bleu_score,meteor_score,bert_score,comet_score,ter_score
0,en_pt_br,So we went over into Michael's garage — his da...,Fomos na garagem de Michael — seu pai tinha to...,Então vamos até o garaje de Michael – o pai de...,548672,SRC_TM_DATA/data_2024/en-pt_br.tmx,0.107790,0.396407,0.985662,0.720391,77.142857
1,en_pt_br,"I look out into this crowd, and I see 400 peop...",Eu olho para essa multidão e vejo 400 pessoas ...,Olho para esta multidão e vejo 400 pessoas que...,121062,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.227517,0.569717,0.988639,0.821432,54.054054
2,en_pt_br,"In fact, part of the charm and the innovation ...",Parte do charme e da inovação do trabalho real...,"Na verdade, parte do charme e da inovação dess...",142335,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.240695,0.592581,0.993191,0.825651,54.929577
3,en_pt_br,So you might be wondering whether we can also ...,Você deve estar perguntando se podemos também ...,Aí você pode estar se perguntando se podemos t...,157346,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.341300,0.532304,0.993841,0.808510,50.000000
4,en_pt_br,But there's also the fact that law is another ...,Mas há também o fato de que a lei é outro luga...,Mas também há o fato de que a lei é outro luga...,76527,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.437664,0.739610,0.989328,0.808980,45.312500
...,...,...,...,...,...,...,...,...,...,...,...
1995,en_pt_br,We're now moving into East Africa.,Agora estamos expandindo para a África Oriental.,Agora estamos mudando para África Oriental.,269204,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.186512,0.712025,0.957964,0.931213,28.571429
1996,en_pt_br,I had just gone through a breakup.,Eu tinha acabado de terminar um namoro.,Eu havia apenas passado por um fim de relacion...,108436,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.029847,0.243902,0.902667,0.611491,100.000000
1997,en_pt_br,Facit fell into the success trap.,A Facit caiu na armadilha do sucesso.,Caiu no trap da sucesso.,200436,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.061861,0.327635,0.916724,0.703556,71.428571
1998,en_pt_br,They looked straight through me.,Olharam direto para mim.,Eles olharam direto em mim.,232428,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.095544,0.735294,0.924132,0.861463,50.000000


In [ ]:
df.drop('Unnamed: 0.1', axis=1)
df.drop('Unnamed: 0', axis=1)
df.to_csv(path)

In [ ]:
df.drop(columns=[col for col in df.columns if 'Unnamed' in col], inplace=True)

In [ ]:
df.to_csv(path)
files.download(path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df

,lang_pair,source,reference,llama2:13b,inner_id,produced_from,bleu_score,meteor_score,bert_score,comet_score,ter_score
0,en_pt_br,So we went over into Michael's garage — his da...,Fomos na garagem de Michael — seu pai tinha to...,"I apologize, but as a professional translator,...",548672,SRC_TM_DATA/data_2024/en-pt_br.tmx,0.002578,0.053508,0.460718,0.345212,148.571429
1,en_pt_br,"I look out into this crowd, and I see 400 peop...",Eu olho para essa multidão e vejo 400 pessoas ...,Eu olho para esta multidão e vejo 400 pessoas ...,121062,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.300942,0.626229,0.988050,0.834073,48.648649
2,en_pt_br,"In fact, part of the charm and the innovation ...",Parte do charme e da inovação do trabalho real...,"Efectivamente, parte da charm e da inovação do...",142335,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.352403,0.674897,0.992952,0.806705,46.478873
3,en_pt_br,So you might be wondering whether we can also ...,Você deve estar perguntando se podemos também ...,Você pode estar wonderendo se podemos fazer es...,157346,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.171031,0.421774,0.986065,0.755989,64.705882
4,en_pt_br,But there's also the fact that law is another ...,Mas há também o fato de que a lei é outro luga...,Mas há também o fato de que a lei é outro luga...,76527,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.524037,0.737832,0.989631,0.828053,35.937500
...,...,...,...,...,...,...,...,...,...,...,...
1995,en_pt_br,We're now moving into East Africa.,Agora estamos expandindo para a África Oriental.,Estamos agora movendo-se para África Oriental.,269204,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.152286,0.646976,0.910449,0.910012,42.857143
1996,en_pt_br,I had just gone through a breakup.,Eu tinha acabado de terminar um namoro.,Eu havia acabado de passar por um término.,108436,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.063767,0.459259,0.928141,0.733967,57.142857
1997,en_pt_br,Facit fell into the success trap.,A Facit caiu na armadilha do sucesso.,Facit caiu na armadilha do sucesso.,200436,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.866878,0.884784,0.993513,0.903326,14.285714
1998,en_pt_br,They looked straight through me.,Olharam direto para mim.,Eles olharam diretamente para mim.,232428,SRC_TM_DATA/data_2024/en-pt_br-2.tmx,0.202052,0.735294,0.936232,0.881574,50.000000
